In [4]:
import numpy as np

# Naive Gaussian Elimination

Three operations applied to a system of linear equations which yield an equivalent system:
1. Swap one equation for another
2. Add or subtract a multiple of one euqation from another
3. Multiply an equation by a non-zero constant

Suppose we have,
$$ x + y = 3 $$
$$ 3x - 4y = 2 $$
by subtracting 3 times the first equation from the second equation
$$ x + y = 3$$
$$ - 7y = - y $$
We can then backsolve to a solution $(x, y) = (2, 1)$.


Foreshadowing Gaussian elimination,

$$
\left[
\begin{array}{cc|c}
 1 & 1 & 3\\
 3 & -4 & 2 \\
\end{array}
\right]
$$
Becomes,
$$
\left[
\begin{array}{cc|c}
 1 & 1 & 3\\
 0 & -7 & -7 \\
\end{array}
\right]
$$
after subtracting 3 times the first row from the first row.

$$
\left[
\begin{array}{cccc|c}
 a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
 a_{21} & a_{22} & \cdots & a_{2n} & b_2 \\
 \vdots & \vdots & \cdots & \vdots & \vdots \\
 a_{n1} & a_{n2} & \cdots & a_{nn} & b_n \\
\end{array}
\right]
$$

**elimination step:** To carry out this step, we need to put zeros in the lower triangular area of the augmented matrix - use row operations to put a zero in each location below the diagonal.
The location sof the main diagonal are then given by $a_{j+1,j}, a_{j+2,j},...,a_{nj}$.

For example, if we want to eliminate column 1, we need to put zeros in $a_{21}, a_{31},...,a_{n1}$.
```
for j in range(n-1):
    for i in range(j+1, n):
        eliminate a[i,j]
```

This begs the question, how do we actually eliminate any entries?
Well, if we wanted to eliminate entry $a_{21}$ we could subtract $a_{21}/a_{11}$ times row 1 from row 2 - just as we did in our first example - assuming the entry right above is not zero. In this case, $a_{11} \ne 0$.

So to get rid of one entry we carry out 1 division, n multiplications, and n additions -> $2n + 1$ operations for a total of $n-1$ times.

After this is done, we choose $a_{22}$ as our next pivot and proceed accordingly.

```
for j in range(n-1):                        # for all columns up to the rightmost column
    assert abs(a[j,j]) > eps                # proceed if pivot is greater than some mun value
    for i in range(j+1, n):                 # for each entry below the diagonal in column j
        mult = a[i,j] / a[j,j]              # multiplying factor, divide entry by pivot
        for k in range(j+1, n):             # for all entries in the given row
            a[i,k] = a[i,k] - mult * a[j,k] 
        b[i] = b[i] - mult * b[j]
```
If $k$ ranged from $j$ to $n$ then we would put a zero in the $a_{ij}$ location.
Though since we will never look back at this entry then this step is simply ignored.


The elimination step for a system of $n$ equations and $n$ variables an be completed in $\frac{2}{3}n^3 + \frac{1}{2}n^2 - \frac{7}{6}n$.

**Nacksubstitution step:** For this step we start at the rightmost-bottom of the matrix and work our way back to the begining.

```
for i in range(n-1,-1,-1):           # Up a colum, from n to 0
    for j in range(i+1, n-1)         # For row
        b[i] = b[i] - a[i,j] * x[j]  
    x[i] = b[i] / a[i,i]
```

This step can be completed in $n^2$ steps.

In [88]:
def naive_gaussian(a, b, eps=1.0e-2):
    n, m = a.shape
    assert n==m
    assert n==b.shape[0]
    
    for j in range(n-1):                        # for all columns up to the rightmost column
        assert abs(a[j,j]) > eps                # proceed if pivot is greater than some mun value
        for i in range(j+1, n):                 # for each entry below the diagonal in column j
            mult = a[i,j] / a[j,j]              # multiplying factor, divide entry by pivot
            for k in range(j+1, n):             # for all entries in the given row
                a[i,k] = a[i,k] - mult * a[j,k] 
            b[i] = b[i] - mult * b[j]
    return a, b
    
def backsubstitution(a, b):
    n = len(b)
    x = [0]*n
    
    for i in range(n-1,-1,-1):         # Up a colum, from n to 0
        for j in range(i+1, n):        # For row
            b[i] = b[i] - a[i,j] * x[j]  
        x[i] = b[i] / a[i,i]
    return x

In [91]:
a = np.array([[1, 1], [3, -4]])
b = np.array([3, 2])
an, bn = naive_gaussian(a, b)
x = backsubstitution(an, bn)
print(an, '\n')
print(bn, '\n')
print(x)

[[ 1  1]
 [ 3 -7]] 

[ 2 -7] 

[2.0, 1.0]


In [92]:
a = np.array([[1, 2, -1], [2, 1, -2], [-3, 1, 1]])
b = np.array([3, 3, -6])
an, bn = naive_gaussian(a, b)
x = backsubstitution(an, bn)
print(an, '\n')
print(bn, '\n')
print(x)

[[ 1  2 -1]
 [ 2 -3  0]
 [-3  7 -2]] 

[ 3 -3 -4] 

[3.0, 1.0, 2.0]


# LU Factorization

Matrix representation of Gaussian elimination. 
It consists of writing the the coefficients matrix $A$ as a product of a lower triangular matrix %L% and an upper triangular matrix $U$.

* An $m \times n$ matrix is a lower triangular matrix if $l_{ij}=0$ for $i<j$.
* An $m \times n$ matrix is an upper triangular matrix if $u_{ij}=0$ for $i>j$.


1. Let $L_{ij} (-c)$ enote the lower triangular matrix whose only nonzero entries are 1s on the diagonal and $-c$ in the $(i,j)$ position. Then $A \rightarrow L_{ij}(-c)A$ represents the row operation "subtracting $c$ times row $j$ from row $i$."

2. $L_{ij}(-c)^{-1} = L_{ij}(c)$

3.
$$
\left[ \begin{array}{ccc}
1   &   & \\
c_1 & 1 & \\
    &   & 1 \\
\end{array} \right] 
\
\left[ \begin{array}{ccc}
 1  &   &  \\
    & 1 &  \\
c_2 & & 1 \\
\end{array} \right]
\
\left[ \begin{array}{ccc}
1 &     & \\
  & 1   & \\
  & c_3 & 1
\end{array} \right]  
\
= \left[ 
    \begin{array}{ccc}
    1   &      & \\
    c_1 & 1    & \\
    c_2 & c_3  & 1
\end{array} \right]
$$

Once $L$ and $U$ are known, then $Ax = LUx = Lc = b$.
Then the backsubstitution step has two steps:
1. Solve $Lc = b$ for $c$.
2. Solve $Ux = c$ for $x$.

The two steps are straightforward since $L$ and $U$ are triangular matrices.


Suppose we need to solve different problems with the same $A$ but with $k$ different $b$'s.
Classical Gaussian elimination will require approximately $kn^3$ operations.
However, using the LU decompositon the $b$ doesn't enter until the elimination step has finished - once the decomposition $A=LU$ is complete.
Thus the number of operations would then be $kn^3 + kn^2$.

In [115]:
def LU_decomposition(a):
    n, m = a.shape
    assert n==m
    assert n==b.shape[0]
    
    L = np.identity(m)
    U = np.copy(a)
    for j in range(n-1):                                       
        for i in range(j+1, n): 
            assert a[j,j] != 0.0
            
            mult = a[i,j] / a[j,j]
            L[i,j] = mult 
            for k in range(j, n): 
                op = a[i,k] - mult * a[j,k]
                a[i,k] = op
                U[i,k] = op
            
    return L, U

In [116]:
a = np.array([[1, 1], [3, -4]])
b = np.array([3, 2])
L, U = LU_decomposition(a)
print(L)
print(U)

[[ 1.  0.]
 [ 3.  1.]]
[[ 1  1]
 [ 0 -7]]


In [117]:
a = np.array([[1, 2, -1], [2, 1, -2], [-3, 1, 1]])
b = np.array([3, 3, -6])
L, U = LU_decomposition(a)
print(L)
print(U)

[[ 1.          0.          0.        ]
 [ 2.          1.          0.        ]
 [-3.         -2.33333333  1.        ]]
[[ 1  2 -1]
 [ 0 -3  0]
 [ 0  0 -2]]
